### Retrieve raw data from GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# verify GPU available
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec 15 01:43:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Download package dependencies


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 7.0 MB/s 
     |████████████████████████████████| 7.6 MB 41.4 MB/s 
     |████████████████████████████████| 182 kB 38.2 MB/s 


In [ ]:
!pip install sentencepiece
# to avoid sentencepiece error, restart runtime and install each separately

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!wget https://raw.githubusercontent.com/itayniv/aesop-fables-stories/master/public/aesopFables.json
# Thanks to Itay Niv for JSON dataset 
# https://github.com/itayniv/aesop-fables-stories/blob/master/public/aesopFables.json
# Text originally from Project Gutenberg: https://www.gutenberg.org/files/49010/49010-0.txt
# Aesop's Fables, by J. H. Stickney

--2022-10-04 20:42:49--  https://raw.githubusercontent.com/itayniv/aesop-fables-stories/master/public/aesopFables.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167691 (164K) [text/plain]
Saving to: ‘aesopFables.json.1’

aesopFables.json.1  100%[===================>] 163.76K  --.-KB/s    in 0.01s   

2022-10-04 20:42:49 (11.8 MB/s) - ‘aesopFables.json.1’ saved [167691/167691]



### Preprocessing datasets



###### Aesop's Fables

In [ ]:
import pandas as pd
import json
dir = 'drive/MyDrive/'
file_path = f'{dir}aesopFables.json'
data = json.load(open(file_path)) # load json string as python dict
df =  pd.DataFrame.from_dict(data['stories']) # unnest, create dataframe
df = df.filter(['story','moral']) # remove unneeded metadata
# convert to strings
#type(df['story'][0]) # because the stories were loaded as type:list
df['story'] = df['story'].transform(lambda s: ' '.join(s))
#type(df['story'][0]) # confirm changes

In [ ]:
df.head() # verify data loaded

,story,moral
0,There was once a little Kid whose growing horn...,Do not let anything turn you from your purpose.
1,"The Tortoise, you know, carries his house on h...",Foolish curiosity and vanity often lead to mis...
2,Why in the world do you walk sideways like tha...,Do not tell others how to act unless you can s...
3,An Ox came down to a reedy pool to drink. As h...,Do not attempt the impossible.
4,"A Dog and a Cock, who were the best of friends...",Those who try to deceive may expect to be paid...


In [ ]:
df.describe() # check size of dataset

,story,moral
count,147,147
unique,147,145
top,There was once a little Kid whose growing horn...,A kindness is never wasted.
freq,1,2


In [ ]:
# check first entry
df['story'][0]

"There was once a little Kid whose growing horns made him think he was a grown-up Billy Goat and able to take care of himself. So one evening when the flock started home from the pasture and his mother called, the Kid paid no heed and kept right on nibbling the tender grass. A little later when he lifted his head, the flock was gone. He was all alone. The sun was sinking. Long shadows came creeping over the ground. A chilly little wind came creeping with them making scary noises in the grass. The Kid shivered as he thought of the terrible Wolf. Then he started wildly over the field, bleating for his mother. But not half-way, near a clump of trees, there was the Wolf! The Kid knew there was little hope for him. Please, Mr. Wolf, he said trembling, I know you are going to eat me. But first please pipe me a tune, for I want to dance and be merry as long as I can. The Wolf liked the idea of a little music before eating, so he struck up a merry tune and the Kid leaped and frisked gaily. Mea

In [ ]:
df['story'].str.len().mean() 
# mean length (in characters) of the stories

837.9183673469388

In [ ]:
df['story'].str.len().max()
# longest text (in characters)

2207

In [ ]:
word_count = df['story'].str.split().str.len()
word_count.mean()
# mean word count

163.93197278911563

In [ ]:
word_count.max()
# longest text (in words)

430

###### Folk Tales Dataset by
[Andrzej Panczenko](https://www.kaggle.com/datasets/andrzejpanczenko/folk-tales-dataset) collected from https://fairytalez.com/regions/

In [ ]:
ft = pd.read_csv('drive/MyDrive/fairytalez.csv')
ft = ft[ft['nation'] != 'greek'] # avoid duplicate aesops
#ft_text = ft.filter(['text']) # remove everything except folktale texts
ft.head()

,source,nation,title,text
0,https://fairytalez.com/momotaro/,japanese,Momotaro,If you’ll believe me there was a time when the...
1,https://fairytalez.com/the-birdcatcher/,serbian,The Birdcatcher,Near Constantinople there lived a man who knew...
2,https://fairytalez.com/sharing-joy-and-sorrow/,german,Sharing Joy and Sorrow,"There was once a tailor, who was a quarrelsome..."
3,https://fairytalez.com/the-punishment-of-gangana/,french,The Punishment of Gangana,Once upon a time there lived a king and queen ...
4,https://fairytalez.com/the-peace-with-the-snakes/,north_american_native,The Peace with the Snakes,In those days there was a Piegan chief named O...


In [ ]:
word_count = ft['text'].str.split().str.len()
word_count.mean()

1828.9472285497343

In [ ]:
ft['text'][0]

'If you’ll believe me there was a time when the fairies were none so shy as they are now. That was the time when beasts talked to men, when there were spells and enchantments and magic every day, when there was great store of hidden treasure to be dug up, and adventures for the asking.At that time, you must know, an old man and an old woman lived alone by themselves. They were good and they were poor and they had no children at all.One fine day, “What are you doing this morning, good man?” says the old woman.“Oh,” says the old man, “I’m off to the mountains with my billhook to gather a faggot of sticks for our fire. And what are you doing, good wife?”“Oh,” says the old woman, “I’m off to the stream to wash clothes. It’s my washing day,” she adds.So the old man went to the mountains and the old woman went to the stream.Now, while she was washing the clothes, what should she see but a fine ripe peach that came floating down the stream? The peach was big enough, and rosy red on both sides

Filter the folktales dataset to retrieve only texts of comparable length to Aesop's fables.

In [ ]:
limit = 400
# Aesop's fables are all under 400 words
ft_short = ft.loc[ft['text'].str.split().str.len() <= limit] # folktale texts under word limit
ft_short.describe()

,source,nation,title,text
count,376,376,376,376
unique,376,33,374,375
top,https://fairytalez.com/sharing-joy-and-sorrow/,french,The Passing of Loku,"The tale of Loku is applied to a large, ugly l..."
freq,1,95,2,2


In [ ]:
word_count = ft_short['text'].str.split().str.len()
word_count.mean()

In [ ]:
ft_short.head()

,source,nation,title,text
2,https://fairytalez.com/sharing-joy-and-sorrow/,german,Sharing Joy and Sorrow,"There was once a tailor, who was a quarrelsome..."
13,https://fairytalez.com/the-peasant-in-heaven/,german,The Peasant in Heaven,"ONCE on a time a poor pious peasant died, and ..."
34,https://fairytalez.com/the-laird-o-co/,scottish,The Laird O' Co',"In the days of yore, the proprietors of Colzea..."
54,https://fairytalez.com/fool-sold-wisdom/,french,The Fool Who Sold Wisdom,"A fool, in town, did wisdom cry;The people, ea..."
55,https://fairytalez.com/quarrel-dogs-cats/,french,Quarrel of the Dogs and Cats,"In mansion deck’d with frieze and column,Dwelt..."


In [ ]:
ft_short['text'].iloc[0]

'There was once a tailor, who was a quarrelsome fellow, and his wife, who was good, industrious, and pious, never could please him. Whatever she did, he was not satisfied, but grumbled and scolded, and knocked her about and beat her. As the authorities at last heard of it, they had him summoned, and put in prison in order to make him better. He was kept for a while on bread and water, and then set free again. He was forced, however, to promise not to beat his wife any more, but to live with her in peace, and share joy and sorrow with her, as married people ought to do. All went on well for a time, but then he fell into his old ways, and was surly and quarrelsome. And because he dared not beat her, he would seize her by the hair and tear it out. The woman escaped from him, and sprang out into the yard, but he ran after her with his yard-measure and scissors, and chased her about, and threw the yard-measure and scissors at her, and whatever else came his way. When he hit her he laughed, 

In [ ]:
len(ft_short['text'].iloc[0].split())

377

<hr>

### Few-Shot Learning with GPT-J


Given a short story with a moral lesson at the end, can the model generate a moral for another text?

In [ ]:
# load model
import requests
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

API_TOKEN = open('drive/MyDrive/hf_token.txt').read() # HuggingFace API token stored in drive for security

def query(payload='',parameters=None,options={'use_cache': False}):
    API_URL = "https://api-inference.huggingface.co/models/EleutherAI/gpt-j-6B" # GPT-J
    headers = {"Authorization": f"Bearer {API_TOKEN}"}
    body = {"inputs":payload,'parameters':parameters,'options':options}
    response = requests.request("POST", API_URL, headers=headers, data= json.dumps(body))
    try:
      response.raise_for_status()
    except requests.exceptions.HTTPError:
        return "Error:"+" ".join(response.json()['error'])
    else:
      return response.json()[0]['generated_text']

parameters = {
    'max_new_tokens':50,  # number of generated tokens
    'temperature': 0.5,   # controlling the randomness of generations
    'end_sequence': "###" # stopping sequence for generation
}


#### Prompt creation

In [ ]:
prompt = f"{df['story'][0]} Moral: {df['moral'][0]} ###"
#prompt = f"{df['story'][0]} The end. What is the moral of the story? Moral: {df['moral'][0]} ###"
data = query(prompt,parameters)

In [ ]:
data

"There was once a little Kid whose growing horns made him think he was a grown-up Billy Goat and able to take care of himself. So one evening when the flock started home from the pasture and his mother called, the Kid paid no heed and kept right on nibbling the tender grass. A little later when he lifted his head, the flock was gone. He was all alone. The sun was sinking. Long shadows came creeping over the ground. A chilly little wind came creeping with them making scary noises in the grass. The Kid shivered as he thought of the terrible Wolf. Then he started wildly over the field, bleating for his mother. But not half-way, near a clump of trees, there was the Wolf! The Kid knew there was little hope for him. Please, Mr. Wolf, he said trembling, I know you are going to eat me. But first please pipe me a tune, for I want to dance and be merry as long as I can. The Wolf liked the idea of a little music before eating, so he struck up a merry tune and the Kid leaped and frisked gaily. Mea

In [ ]:
prompt = ""
n = 3 # 6,5,4 gave out of memory error (even with Colab Pro GPU runtime)
for i in range(n):
    prompt += f"{df['story'][i]}  Moral: {df['moral'][i]} ### "
    if i == n-1:
        prompt += f"{df['story'][n]}  Moral: " # leave lesson blank for GPT-J to generate
    #    prompt += f"{ft['text'][0]}  Moral: " # concatenate non-Aesop at the end
prompt

"There was once a little Kid whose growing horns made him think he was a grown-up Billy Goat and able to take care of himself. So one evening when the flock started home from the pasture and his mother called, the Kid paid no heed and kept right on nibbling the tender grass. A little later when he lifted his head, the flock was gone. He was all alone. The sun was sinking. Long shadows came creeping over the ground. A chilly little wind came creeping with them making scary noises in the grass. The Kid shivered as he thought of the terrible Wolf. Then he started wildly over the field, bleating for his mother. But not half-way, near a clump of trees, there was the Wolf! The Kid knew there was little hope for him. Please, Mr. Wolf, he said trembling, I know you are going to eat me. But first please pipe me a tune, for I want to dance and be merry as long as I can. The Wolf liked the idea of a little music before eating, so he struck up a merry tune and the Kid leaped and frisked gaily. Mea

In [ ]:
data = query(prompt, parameters)
data

"There was once a little Kid whose growing horns made him think he was a grown-up Billy Goat and able to take care of himself. So one evening when the flock started home from the pasture and his mother called, the Kid paid no heed and kept right on nibbling the tender grass. A little later when he lifted his head, the flock was gone. He was all alone. The sun was sinking. Long shadows came creeping over the ground. A chilly little wind came creeping with them making scary noises in the grass. The Kid shivered as he thought of the terrible Wolf. Then he started wildly over the field, bleating for his mother. But not half-way, near a clump of trees, there was the Wolf! The Kid knew there was little hope for him. Please, Mr. Wolf, he said trembling, I know you are going to eat me. But first please pipe me a tune, for I want to dance and be merry as long as I can. The Wolf liked the idea of a little music before eating, so he struck up a merry tune and the Kid leaped and frisked gaily. Mea

In [ ]:
df['story'][n]

'An Ox came down to a reedy pool to drink. As he splashed heavily into the water, he crushed a young Frog into the mud. The old Frog soon missed the little one and asked his brothers and sisters what had become of him. A great big monster, said one of them, stepped on little brother with one of his huge feet! Big, was he! said the old Frog, puffing herself up. Was he as big as this? Oh, much bigger! they cried. The Frog puffed up still more. He could not have been bigger than this, she said. But the little Frogs all declared that the monster was much, much bigger and the old Frog kept puffing herself out more and more until, all at once, she burst.'

Prompt size: 3 stories [n = 3]

GPT-J generated: 

    Moral:  Do not puff yourself up.

    Moral: Do not puff yourself up, no matter how big you think you are. 

In [ ]:
# Actual
df['moral'][n]

'Do not attempt the impossible.'

In [ ]:
prompt = ""
n = 1 # non-Aesop is too big, try just one-shot
for i in range(n):
    prompt += f"{df['story'][i]}  Moral: {df['moral'][i]} ### "
    if i == n-1:
    #    prompt += f"{df['story'][n]}  Moral: " # leave lesson blank for GPT-J to generate
        prompt += f"{ft['text'][0]}  Moral: " # concatenate non-Aesop at the end
prompt

"There was once a little Kid whose growing horns made him think he was a grown-up Billy Goat and able to take care of himself. So one evening when the flock started home from the pasture and his mother called, the Kid paid no heed and kept right on nibbling the tender grass. A little later when he lifted his head, the flock was gone. He was all alone. The sun was sinking. Long shadows came creeping over the ground. A chilly little wind came creeping with them making scary noises in the grass. The Kid shivered as he thought of the terrible Wolf. Then he started wildly over the field, bleating for his mother. But not half-way, near a clump of trees, there was the Wolf! The Kid knew there was little hope for him. Please, Mr. Wolf, he said trembling, I know you are going to eat me. But first please pipe me a tune, for I want to dance and be merry as long as I can. The Wolf liked the idea of a little music before eating, so he struck up a merry tune and the Kid leaped and frisked gaily. Mea

In [ ]:
len(prompt.split())

1263

In [ ]:
data = query(prompt,parameters)
data

'Error:I n p u t   i s   t o o   l o n g ,   t r y   t o   t r u n c a t e   o r   u s e   a   p a r a m a t e r   t o   h a n d l e   t h i s :   T h e   s i z e   o f   t e n s o r   a   ( 2 0 4 8 )   m u s t   m a t c h   t h e   s i z e   o f   t e n s o r   b   ( 2 0 8 3 )   a t   n o n - s i n g l e t o n   d i m e n s i o n   3'

#### Generating morals for all Aesop's Fables



In [ ]:
# make a copy of the df so nothing weird happens if we make changes while iterating over the original
output = df
output['moral_generated'] = ''
output['full_response'] = ''
output.head()

,story,moral,moral_generated,full_response
0,There was once a little Kid whose growing horn...,Do not let anything turn you from your purpose.,,
1,"The Tortoise, you know, carries his house on h...",Foolish curiosity and vanity often lead to mis...,,
2,Why in the world do you walk sideways like tha...,Do not tell others how to act unless you can s...,,
3,An Ox came down to a reedy pool to drink. As h...,Do not attempt the impossible.,,
4,"A Dog and a Cock, who were the best of friends...",Those who try to deceive may expect to be paid...,,


In [ ]:
from tqdm import tqdm # progress bar. use: tqdm(iterable)
import re
fables = len(df) # 147 fables # 20~30 minutes
#fables = 5 # TEST
prefix = "Moral:"
pattern = r"Moral:(.*?)###"
for story in tqdm(range(fables)):
    #print(f"\nSTORY #{story}")
    prompt = ""
    prompt += f"{df['story'].iloc[story-3]}  {prefix} {df['moral'].iloc[story-3]} ### "
    prompt += f"{df['story'].iloc[story-2]}  {prefix} {df['moral'].iloc[story-2]} ### "
    prompt += f"{df['story'].iloc[story-1]}  {prefix} {df['moral'].iloc[story-1]} ### "
    prompt += f"{df['story'].iloc[story]}  {prefix} " # leave lesson blank for GPT-J to generate
    #print("PROMPT\n",prompt)
    generated = query(prompt, parameters)
    #print("RESPONSE\n",generated[-500:])
    moral = re.search(pattern, generated[-500:]) # search the last 500 characters (to avoid original prompt) (+speeds up regex searching by ~10 min!)
    if moral:
        print("\nMORAL:",moral.group(1))
        #moral = match.groups()[-1] # groups() returns tuple, -1 is the last match # 0 is the search pattern
        output['moral_generated'][story] = moral.group(1)
        output['full_response'][story] = generated
    print("~"*77)

  1%|          | 1/147 [00:09<24:14,  9.96s/it]


MORAL:   A fool and his money are soon parted. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


  1%|▏         | 2/147 [00:15<17:32,  7.26s/it]


MORAL:   The best of friends may disagree. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


  2%|▏         | 3/147 [00:20<15:25,  6.43s/it]

...


MORAL:   
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


 99%|█████████▉| 146/147 [16:33<00:06,  6.60s/it]


MORAL:   The world is full of little fishes who are much better than nothing. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


100%|██████████| 147/147 [16:38<00:00,  6.79s/it]


MORAL:   If you crow too loud, you may be crowing to your own undoing. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [ ]:
output.head(10)

,story,moral,moral_generated,full_response
0,There was once a little Kid whose growing horn...,Do not let anything turn you from your purpose.,A fool and his money are soon parted.,An Ass found a Lion's skin left in the forest ...
1,"The Tortoise, you know, carries his house on h...",Foolish curiosity and vanity often lead to mis...,The best of friends may disagree.,"A poor Fisherman, who lived on the fish he cau..."
2,Why in the world do you walk sideways like tha...,Do not tell others how to act unless you can s...,"""How to walk"" is a great mystery.",Once there were two Cocks living in the same f...
3,An Ox came down to a reedy pool to drink. As h...,Do not attempt the impossible.,Do not puff yourself up.,There was once a little Kid whose growing horn...
4,"A Dog and a Cock, who were the best of friends...",Those who try to deceive may expect to be paid...,Do not trust a fox who has a good breakfast ...,"The Tortoise, you know, carries his house on h..."
5,The Mice once called a meeting to decide on a ...,It is one thing to say that something should b...,Don't be too hasty in your judgment of others.,Why in the world do you walk sideways like tha...
6,"An Eagle, swooping down on powerful wings, sei...",Do not let your vanity make you overestimate y...,Do not boast of your own strength or abiliti...,An Ox came down to a reedy pool to drink. As h...
7,A Boy was given permission to put his hand int...,Do not attempt too much at once.,"Do not be a greedy child, but a satisfied one.","A Dog and a Cock, who were the best of friends..."
8,A Farmer was driving his wagon along a miry co...,Self help is the best help.,"When you are in trouble, do something.",The Mice once called a meeting to decide on a ...
9,A frisky young Kid had been left by the herdsm...,Do not say anything at any time that you would...,Do not be too quick to take offence.,"An Eagle, swooping down on powerful wings, sei..."


In [ ]:
output['moral'][3]

'Do not attempt the impossible.'

In [ ]:
output['moral_generated'][3]

'  Do not puff yourself up.  '

In [ ]:
#output_csv = output.to_csv('aesop_gptj.csv')

#### Why did GPT-J skip 38 entries?

In [ ]:
import pandas as pd
aesop_gptj = pd.read_csv('drive/MyDrive/aesop_gptj.csv')

##### Investigate entries where the model did not generate any moral

In [ ]:
rerun = aesop_gptj[aesop_gptj.isnull().any(axis=1)]
rerun

,Unnamed: 0,story,moral,moral_generated,full_response
12,12,"A certain Father had a family of Sons, who wer...",In unity is strength.,NaN,A frisky young Kid had been left by the herdsm...
21,21,One day a shepherd discovered a fat Pig in the...,It is easy to be brave when there is no danger.,NaN,A Gnat flew over the meadow with much buzzing ...
25,25,The Owl always takes her sleep during the day....,Flattery is not a proof of true admiration.,NaN,NaN
27,27,A Giant Oak stood near a brook in which grew s...,"Better to yield when it is folly to resist, th...",NaN,NaN
37,37,A certain man who visited foreign lands could ...,"Deeds count, not boasting words.",NaN,"Two Goats, frisking gayly on the rocky steeps ..."
39,39,"At a great celebration in honor of King Lion, ...",Do not try to ape your betters.,NaN,"Away with you, vile insect!, said a Lion angri..."
41,41,"An Ass and a Fox had become close comrades, an...",Traitors may expect treachery.,NaN,A Cock was busily scratching and scraping abou...
45,45,A Wolf had been hurt in a fight with a Bear. H...,A knave's hypocrisy is easily seen through.,NaN,The Birds and the Beasts declared war against ...
56,56,A Dog had learned to carry his master's dinner...,Do not stop to argue with temptation.,NaN,NaN
58,58,It happened once upon a time that a certain Gr...,One falsehood leads to another.,NaN,A Lion and an Ass agreed to go hunting togethe...


In [ ]:
rerun.reset_index(drop=True,inplace=True) # reset index to start at 0 again; original index preserved in column 0
rerun.head()

,Unnamed: 0,story,moral,moral_generated,full_response
0,12,"A certain Father had a family of Sons, who wer...",In unity is strength.,NaN,A frisky young Kid had been left by the herdsm...
1,21,One day a shepherd discovered a fat Pig in the...,It is easy to be brave when there is no danger.,NaN,A Gnat flew over the meadow with much buzzing ...
2,25,The Owl always takes her sleep during the day....,Flattery is not a proof of true admiration.,NaN,NaN
3,27,A Giant Oak stood near a brook in which grew s...,"Better to yield when it is folly to resist, th...",NaN,NaN
4,37,A certain man who visited foreign lands could ...,"Deeds count, not boasting words.",NaN,"Two Goats, frisking gayly on the rocky steeps ..."


In [ ]:
rerun.count()

Unnamed: 0         38
story              38
moral              38
moral_generated     0
full_response      15
dtype: int64

In [ ]:
df = rerun

output = df
output['moral_generated'] = ''
output['full_response'] = ''
output.head()

from tqdm import tqdm 
import re
import json

fables = len(df) 
prefix = "Moral:"
pattern = r"Moral:(.*?)###"
for story in tqdm(range(fables)):
    print("\nSTORY",story)
    prompt = ""
    prompt += f"{df['story'].iloc[story-3]}  {prefix} {df['moral'].iloc[story-3]} ### "
    prompt += f"{df['story'].iloc[story-2]}  {prefix} {df['moral'].iloc[story-2]} ### "
    prompt += f"{df['story'].iloc[story-1]}  {prefix} {df['moral'].iloc[story-1]} ### "
    prompt += f"{df['story'].iloc[story]}  {prefix} " # leave lesson blank for GPT-J to generate
    print(df['story'].iloc[story][:50])
    generated = query(prompt, parameters)
    moral = re.search(pattern, generated[-500:]) # search the last 500 characters (to avoid original prompt) (+speeds up regex searching by ~10 min!)
    if moral:
        print("\nMORAL:",moral.group(1))
        #moral = match.groups()[-1] # groups() returns tuple, -1 is the last match # 0 is the search pattern
        output['moral_generated'][story] = moral.group(1)
        output['full_response'][story] = generated
    print("~"*77)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
  0%|          | 0/38 [00:00<?, ?it/s]


STORY 0
A certain Father had a family of Sons, who were fo


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  3%|▎         | 1/38 [00:05<03:19,  5.38s/it]


MORAL:   If you are united, you will be strong; but if you are divided, you will be weak. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 1
One day a shepherd discovered a fat Pig in the mea


  5%|▌         | 2/38 [00:14<04:26,  7.41s/it]

...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 36
A Dove saw an Ant fall into a brook. The Ant strug


 97%|█████████▋| 37/38 [03:54<00:02,  2.88s/it]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 37
An Ass found a Lion's skin left in the forest by a


100%|██████████| 38/38 [03:59<00:00,  6.30s/it]


MORAL:   Silence is golden. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [ ]:
#output_csv = output.to_csv('aesop_gptj_rerun.csv')

### Combine rerun with first output

In [ ]:
import pandas as pd
rerun = pd.read_csv('drive/MyDrive/aesop_gptj_rerun.csv')
df = pd.read_csv('drive/MyDrive/aesop_gptj.csv')

In [ ]:
#rerun = rerun.set_index('Unnamed: 0.1')
rerun.drop('Unnamed: 0', axis=1, inplace=True)
rerun.head()

,story,moral,moral_generated,full_response
Unnamed: 0.1,,,,
12,"A certain Father had a family of Sons, who wer...",In unity is strength.,"If you are united, you will be strong; but i...","One day, a long time ago, an old Miller and hi..."
21,One day a shepherd discovered a fat Pig in the...,It is easy to be brave when there is no danger.,,A Dove saw an Ant fall into a brook. The Ant s...
25,The Owl always takes her sleep during the day....,Flattery is not a proof of true admiration.,If you are too eager to make friends with a ...,An Ass found a Lion's skin left in the forest ...
27,A Giant Oak stood near a brook in which grew s...,"Better to yield when it is folly to resist, th...",NaN,NaN
37,A certain man who visited foreign lands could ...,"Deeds count, not boasting words.",No need of witnesses when you have the truth...,One day a shepherd discovered a fat Pig in the...


In [ ]:
import numpy as np
rerun.index.name = None # remove index header name
rerun = rerun.replace(r'^\s+$', np.nan, regex=True) # normalize empty whitespace to NaNs
rerun.head()

,story,moral,moral_generated,full_response
12,"A certain Father had a family of Sons, who wer...",In unity is strength.,"If you are united, you will be strong; but i...","One day, a long time ago, an old Miller and hi..."
21,One day a shepherd discovered a fat Pig in the...,It is easy to be brave when there is no danger.,NaN,A Dove saw an Ant fall into a brook. The Ant s...
25,The Owl always takes her sleep during the day....,Flattery is not a proof of true admiration.,If you are too eager to make friends with a ...,An Ass found a Lion's skin left in the forest ...
27,A Giant Oak stood near a brook in which grew s...,"Better to yield when it is folly to resist, th...",NaN,NaN
37,A certain man who visited foreign lands could ...,"Deeds count, not boasting words.",No need of witnesses when you have the truth...,One day a shepherd discovered a fat Pig in the...


In [ ]:
df.fillna(rerun, axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head(50)

,story,moral,moral_generated,full_response
0,There was once a little Kid whose growing horn...,Do not let anything turn you from your purpose.,A fool and his money are soon parted.,An Ass found a Lion's skin left in the forest ...
1,"The Tortoise, you know, carries his house on h...",Foolish curiosity and vanity often lead to mis...,The best of friends may disagree.,"A poor Fisherman, who lived on the fish he cau..."
2,Why in the world do you walk sideways like tha...,Do not tell others how to act unless you can s...,"""How to walk"" is a great mystery.",Once there were two Cocks living in the same f...
3,An Ox came down to a reedy pool to drink. As h...,Do not attempt the impossible.,Do not puff yourself up.,There was once a little Kid whose growing horn...
4,"A Dog and a Cock, who were the best of friends...",Those who try to deceive may expect to be paid...,Do not trust a fox who has a good breakfast in...,"The Tortoise, you know, carries his house on h..."
5,The Mice once called a meeting to decide on a ...,It is one thing to say that something should b...,Don't be too hasty in your judgment of others.,Why in the world do you walk sideways like tha...
6,"An Eagle, swooping down on powerful wings, sei...",Do not let your vanity make you overestimate y...,Do not boast of your own strength or abilities.,An Ox came down to a reedy pool to drink. As h...
7,A Boy was given permission to put his hand int...,Do not attempt too much at once.,"Do not be a greedy child, but a satisfied one.","A Dog and a Cock, who were the best of friends..."
8,A Farmer was driving his wagon along a miry co...,Self help is the best help.,"When you are in trouble, do something.",The Mice once called a meeting to decide on a ...
9,A frisky young Kid had been left by the herdsm...,Do not say anything at any time that you would...,Do not be too quick to take offence.,"An Eagle, swooping down on powerful wings, sei..."


In [ ]:
still_empty = df[df.isnull().any(axis=1)]
still_empty.count()

story              13
moral              13
moral_generated     0
full_response       5
dtype: int64

In [ ]:
output_csv = df.to_csv('aesop_gptj_combined.csv')
still_empty_csv = still_empty.to_csv('aesop_gptj_empty.csv')

#### Increase temperature
Model is still not generating any response at all for some, so try increasing the randomness

In [ ]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/aesop_gptj_empty.csv')

In [ ]:
# load model
import requests
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

API_TOKEN = open('drive/MyDrive/hf_token.txt').read() # HuggingFace API token stored in drive for security

def query(payload='',parameters=None,options={'use_cache': False}):
    API_URL = "https://api-inference.huggingface.co/models/EleutherAI/gpt-j-6B" # GPT-J
    headers = {"Authorization": f"Bearer {API_TOKEN}"}
    body = {"inputs":payload,'parameters':parameters,'options':options}
    response = requests.request("POST", API_URL, headers=headers, data= json.dumps(body))
    try:
      response.raise_for_status()
    except requests.exceptions.HTTPError:
        return "Error:"+" ".join(response.json()['error'])
    else:
      return response.json()[0]['generated_text']

parameters = {
    'max_new_tokens':50,  # number of generated tokens
    'temperature': 0.7,   # controlling the randomness of generations <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< increased to 0.7
    'end_sequence': "###" # stopping sequence for generation
}


In [ ]:
output = df
output['moral_generated'] = ''
output['full_response'] = ''
output.head()

from tqdm import tqdm 
import re
import json

fables = len(df) 
prefix = "Moral:"
pattern = r"Moral:(.*?)###"
for story in tqdm(range(fables)):
    print("\nSTORY",story)
    prompt = ""
    prompt += f"{df['story'].iloc[story-3]}  {prefix} {df['moral'].iloc[story-3]} ### "
    prompt += f"{df['story'].iloc[story-2]}  {prefix} {df['moral'].iloc[story-2]} ### "
    prompt += f"{df['story'].iloc[story-1]}  {prefix} {df['moral'].iloc[story-1]} ### "
    prompt += f"{df['story'].iloc[story]}  {prefix} " # leave lesson blank for GPT-J to generate
    print(df['story'].iloc[story][:50])
    generated = query(prompt, parameters)
    moral = re.search(pattern, generated[-500:]) # search the last 500 characters (to avoid original prompt) (+speeds up regex searching by ~10 min!)
    if moral:
        print("\nMORAL:",moral.group(1))
        #moral = match.groups()[-1] # groups() returns tuple, -1 is the last match # 0 is the search pattern
        output['moral_generated'][story] = moral.group(1)
        output['full_response'][story] = generated
    print("~"*77)

  0%|          | 0/13 [00:00<?, ?it/s]


STORY 0
One day a shepherd discovered a fat Pig in the mea


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  8%|▊         | 1/13 [00:12<02:29, 12.45s/it]


MORAL:   The greatest evil is not always the greatest pain. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 1
A Giant Oak stood near a brook in which grew some 


 15%|█▌        | 2/13 [00:26<02:26, 13.30s/it]

...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 11
One day, a long time ago, an old Miller and his So


 92%|█████████▏| 12/13 [02:00<00:18, 18.61s/it]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 12
A Dove saw an Ant fall into a brook. The Ant strug


100%|██████████| 13/13 [02:09<00:00,  9.96s/it]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [ ]:
df.head(13)

,Unnamed: 0,story,moral,moral_generated,full_response
0,21,One day a shepherd discovered a fat Pig in the...,It is easy to be brave when there is no danger.,The greatest evil is not always the greatest...,A Traveler had hired an Ass to carry him to a ...
1,27,A Giant Oak stood near a brook in which grew s...,"Better to yield when it is folly to resist, th...","The Giant Oak has fallen, and the Reeds bend...","One day, a long time ago, an old Miller and hi..."
2,41,"An Ass and a Fox had become close comrades, an...",Traitors may expect treachery.,Do not trust a friend who is always plotting...,A Dove saw an Ant fall into a brook. The Ant s...
3,58,It happened once upon a time that a certain Gr...,One falsehood leads to another.,"Dogs are loyal, but they are not always inte...",One day a shepherd discovered a fat Pig in the...
4,73,"A very young Mouse, who had never seen anythin...",Do not trust alone to outward appearances.,,
5,93,A Serpent had succeeded in surprising an Eagle...,An act of kindness is well repaid.,,
6,105,"A Fox, swimming across a river, was barely abl...",Better to bear a lesser evil than to risk a gr...,"Be careful of what you wish for, for you may...",It happened once upon a time that a certain Gr...
7,127,A Lark made her nest in a field of young wheat...,Self-help is the best help.,"If you depend on others for your labor, you ...","A very young Mouse, who had never seen anythin..."
8,128,"There was once a Cat who was so watchful, that...",The wise do not let themselves be tricked a se...,When you have to deal with a cunning and tre...,A Serpent had succeeded in surprising an Eagle...
9,129,One bright morning as the Fox was following hi...,The flatterer lives at the expense of those wh...,,


In [ ]:
import numpy as np
df = df.replace(r'^\s+$', np.nan, regex=True) # normalize empty whitespace to NaNs # idk why this didn't work this time
still_empty = df[df.isnull().any(axis=1)]
still_empty.count()

Unnamed: 0         0
story              0
moral              0
moral_generated    0
full_response      0
dtype: int64

In [ ]:
df

,Unnamed: 0,story,moral,moral_generated,full_response
0,21,One day a shepherd discovered a fat Pig in the...,It is easy to be brave when there is no danger.,The greatest evil is not always the greatest...,A Traveler had hired an Ass to carry him to a ...
1,27,A Giant Oak stood near a brook in which grew s...,"Better to yield when it is folly to resist, th...","The Giant Oak has fallen, and the Reeds bend...","One day, a long time ago, an old Miller and hi..."
2,41,"An Ass and a Fox had become close comrades, an...",Traitors may expect treachery.,Do not trust a friend who is always plotting...,A Dove saw an Ant fall into a brook. The Ant s...
3,58,It happened once upon a time that a certain Gr...,One falsehood leads to another.,"Dogs are loyal, but they are not always inte...",One day a shepherd discovered a fat Pig in the...
4,73,"A very young Mouse, who had never seen anythin...",Do not trust alone to outward appearances.,,
5,93,A Serpent had succeeded in surprising an Eagle...,An act of kindness is well repaid.,,
6,105,"A Fox, swimming across a river, was barely abl...",Better to bear a lesser evil than to risk a gr...,"Be careful of what you wish for, for you may...",It happened once upon a time that a certain Gr...
7,127,A Lark made her nest in a field of young wheat...,Self-help is the best help.,"If you depend on others for your labor, you ...","A very young Mouse, who had never seen anythin..."
8,128,"There was once a Cat who was so watchful, that...",The wise do not let themselves be tricked a se...,When you have to deal with a cunning and tre...,A Serpent had succeeded in surprising an Eagle...
9,129,One bright morning as the Fox was following hi...,The flatterer lives at the expense of those wh...,,


In [ ]:
still_empty_csv = still_empty.to_csv('aesop_gptj_empty_after_temp7.csv')

In [ ]:
# load model
import requests
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

API_TOKEN = open('drive/MyDrive/hf_token.txt').read() # HuggingFace API token stored in drive for security

def query(payload='',parameters=None,options={'use_cache': False}):
    API_URL = "https://api-inference.huggingface.co/models/EleutherAI/gpt-j-6B" # GPT-J
    headers = {"Authorization": f"Bearer {API_TOKEN}"}
    body = {"inputs":payload,'parameters':parameters,'options':options}
    response = requests.request("POST", API_URL, headers=headers, data= json.dumps(body))
    try:
      response.raise_for_status()
    except requests.exceptions.HTTPError:
        return "Error:"+" ".join(response.json()['error'])
    else:
      return response.json()[0]['generated_text']

parameters = {
    'max_new_tokens':50,  # number of generated tokens
    'temperature': 0.8,   # controlling the randomness of generations <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< increased to 0.8
    'end_sequence': "###" # stopping sequence for generation
}


In [ ]:
empty = [21, 41, 58, 73, 93, 105, 127, 128, 129, 130, 131, 144]
output = df
output['moral_generated'] = ''
output['full_response'] = ''
output.head()

from tqdm import tqdm 
import re
import json

prefix = "Moral:"
pattern = r"Moral:(.*?)###"
for story in tqdm(empty):
    print("\nSTORY",story)
    prompt = ""
    prompt += f"{df['story'].iloc[story-3]}  {prefix} {df['moral'].iloc[story-3]} ### "
    prompt += f"{df['story'].iloc[story-2]}  {prefix} {df['moral'].iloc[story-2]} ### "
    prompt += f"{df['story'].iloc[story-1]}  {prefix} {df['moral'].iloc[story-1]} ### "
    prompt += f"{df['story'].iloc[story]}  {prefix} " # leave lesson blank for GPT-J to generate
    print(df['story'].iloc[story][:50])
    generated = query(prompt, parameters)
    moral = re.search(pattern, generated[-500:]) # search the last 500 characters (to avoid original prompt) (+speeds up regex searching by ~10 min!)
    if moral:
        print("\nMORAL:",moral.group(1))
        #moral = match.groups()[-1] # groups() returns tuple, -1 is the last match # 0 is the search pattern
        output['moral_generated'][story] = moral.group(1)
        output['full_response'][story] = generated
    print("~"*77)

  0%|          | 0/12 [00:00<?, ?it/s]


STORY 21
One day a shepherd discovered a fat Pig in the mea


  8%|▊         | 1/12 [00:04<00:45,  4.12s/it]


MORAL:   
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 41
An Ass and a Fox had become close comrades, and we


 17%|█▋        | 2/12 [00:08<00:42,  4.21s/it]

...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 131
One day, a long time ago, an old Miller and his So


 92%|█████████▏| 11/12 [00:52<00:03,  3.41s/it]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 144
An Ass found a Lion's skin left in the forest by a


100%|██████████| 12/12 [00:58<00:00,  4.91s/it]


MORAL:   Those with horns boast of their horns, and flatter their enemies with praise. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [ ]:
out = output.to_csv('aesop_gptj_temp8.csv')

#### Randomize prompt
It appears some of the fables may be unsuitable for prompting, possibly due to short length
* Randomly select which story to build the prompt from

In [ ]:

empty = [21, 27, 58, 93, 127, 126, 128, 129, 130, 131, 132]
output = df
output['moral_generated'] = ''
output['full_response'] = ''
output.head()

from tqdm import tqdm 
import re
import json
import random

prefix = "Moral:"
pattern = r"Moral:(.*?)###"
for story in tqdm(empty):
    n = random.randint(1,147)
    print("randomized prompt seed", n)
    print("\nSTORY",story)
    prompt = ""
    prompt += f"{df['story'].iloc[n-3]}  {prefix} {df['moral'].iloc[story-3]} ### "
    prompt += f"{df['story'].iloc[n-2]}  {prefix} {df['moral'].iloc[story-2]} ### "
    prompt += f"{df['story'].iloc[n-1]}  {prefix} {df['moral'].iloc[story-1]} ### "
    prompt += f"{df['story'].iloc[story]}  {prefix} " # leave lesson blank for GPT-J to generate
    print(df['story'].iloc[story][:50])
    generated = query(prompt, parameters)
    moral = re.search(pattern, generated[-500:]) # search the last 500 characters (to avoid original prompt) (+speeds up regex searching by ~10 min!)
    if moral:
        print("\nMORAL:",moral.group(1))
        #moral = match.groups()[-1] # groups() returns tuple, -1 is the last match # 0 is the search pattern
        output['moral_generated'][story] = moral.group(1)
        output['full_response'][story] = generated
    print("~"*77)

  0%|          | 0/11 [00:00<?, ?it/s]

randomized prompt seed 125

STORY 21
One day a shepherd discovered a fat Pig in the mea


  9%|▉         | 1/11 [00:05<00:52,  5.22s/it]


MORAL:   If anyone wants to get along with us, he must be of our own kind.  
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
randomized prompt seed 88

STORY 27
A Giant Oak stood near a brook in which grew some 


 18%|█▊        | 2/11 [00:09<00:41,  4.64s/it]

...


MORAL:   If they are not willing to pay for what they receive, they should not be given anything. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
randomized prompt seed 116

STORY 131
One day, a long time ago, an old Miller and his So


 91%|█████████ | 10/11 [01:01<00:06,  6.67s/it]


MORAL:   "A Fool may get a good deal of good out of an Ass but an Ass will have little to do with a Fool." 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
randomized prompt seed 109

STORY 132
A Dove saw an Ant fall into a brook. The Ant strug


100%|██████████| 11/11 [01:05<00:00,  5.98s/it]


MORAL:   We should do more to save the life of an animal than to save our own. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [ ]:
out = output.to_csv('aesop_gptj_temp8_randomizedpromptseed.csv')

Model was able to generate a response for ALL of the empties after randomizing the prompt seed!

Not all Aesops are created equal: Some of the stories aren't as good at priming GPT-J to output a moral.

Go back and see if we can generate them at the original temperature when the seed is randomized.

In [ ]:
# load model
import requests
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

API_TOKEN = open('drive/MyDrive/hf_token.txt').read() # HuggingFace API token stored in drive for security

def query(payload='',parameters=None,options={'use_cache': False}):
    API_URL = "https://api-inference.huggingface.co/models/EleutherAI/gpt-j-6B" # GPT-J
    headers = {"Authorization": f"Bearer {API_TOKEN}"}
    body = {"inputs":payload,'parameters':parameters,'options':options}
    response = requests.request("POST", API_URL, headers=headers, data= json.dumps(body))
    try:
      response.raise_for_status()
    except requests.exceptions.HTTPError:
        return "Error:"+" ".join(response.json()['error'])
    else:
      return response.json()[0]['generated_text']

parameters = {
    'max_new_tokens':50,  # number of generated tokens
    'temperature': 0.7,   # controlling the randomness of generations <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 
    'end_sequence': "###" # stopping sequence for generation
}


In [ ]:
empty = [21, 27, 58, 93, 127, 126, 128, 129, 130, 131, 132]
output = df
output['moral_generated'] = ''
output['full_response'] = ''
output.head()

from tqdm import tqdm 
import re
import json
import random

prefix = "Moral:"
pattern = r"Moral:(.*?)###"
for story in tqdm(empty):
    n = random.randint(1,147)
    print("randomized prompt seed", n)
    print("\nSTORY",story)
    prompt = ""
    prompt += f"{df['story'].iloc[n-3]}  {prefix} {df['moral'].iloc[story-3]} ### "
    prompt += f"{df['story'].iloc[n-2]}  {prefix} {df['moral'].iloc[story-2]} ### "
    prompt += f"{df['story'].iloc[n-1]}  {prefix} {df['moral'].iloc[story-1]} ### "
    prompt += f"{df['story'].iloc[story]}  {prefix} " # leave lesson blank for GPT-J to generate
    print(df['story'].iloc[story][:50])
    generated = query(prompt, parameters)
    moral = re.search(pattern, generated[-500:]) # search the last 500 characters (to avoid original prompt) (+speeds up regex searching by ~10 min!)
    if moral:
        print("\nMORAL:",moral.group(1))
        #moral = match.groups()[-1] # groups() returns tuple, -1 is the last match # 0 is the search pattern
        output['moral_generated'][story] = moral.group(1)
        output['full_response'][story] = generated
    print("~"*77)

  0%|          | 0/11 [00:00<?, ?it/s]

randomized prompt seed 77

STORY 21
One day a shepherd discovered a fat Pig in the mea


  9%|▉         | 1/11 [00:02<00:20,  2.01s/it]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
randomized prompt seed 58

STORY 27
A Giant Oak stood near a brook in which grew some 


 18%|█▊        | 2/11 [00:06<00:30,  3.42s/it]

...


MORAL:   A man's first duty is to himself. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
randomized prompt seed 22

STORY 131
One day, a long time ago, an old Miller and his So


 91%|█████████ | 10/11 [00:53<00:05,  5.52s/it]


MORAL:   A man who does not know how to serve himself is hardly fit to serve his master. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
randomized prompt seed 83

STORY 132
A Dove saw an Ant fall into a brook. The Ant strug


100%|██████████| 11/11 [00:57<00:00,  5.25s/it]


MORAL:   Never try to do a kindness to a scoundrel. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Nope, it still struggles with some of them.

In [ ]:
out = output.to_csv('aeops_gptj_temp5_randomizedpromptseed.csv')

### Prompt with fable + folktale

In [ ]:
# problematic prompts were less than 100 words
len(df['story'].iloc[136].split())

97

In [ ]:
# remove the fables that are too short
df = df.loc[df['story'].str.split().str.len() >= 100]
df.describe()

,story,moral
count,118,118
unique,118,117
top,There was once a little Kid whose growing horn...,In unity is strength.
freq,1,2


In [ ]:
# load model
import requests
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

API_TOKEN = open('drive/MyDrive/hf_token.txt').read() # HuggingFace API token stored in drive for security

def query(payload='',parameters=None,options={'use_cache': False}):
    API_URL = "https://api-inference.huggingface.co/models/EleutherAI/gpt-j-6B" # GPT-J
    headers = {"Authorization": f"Bearer {API_TOKEN}"}
    body = {"inputs":payload,'parameters':parameters,'options':options}
    response = requests.request("POST", API_URL, headers=headers, data= json.dumps(body))
    try:
      response.raise_for_status()
    except requests.exceptions.HTTPError:
        return "Error:"+" ".join(response.json()['error'])
    else:
      return response.json()[0]['generated_text']

parameters = {
    'max_new_tokens':50,  # number of generated tokens
    'temperature': 0.5,   # controlling the randomness of generations
    'end_sequence': "###" # stopping sequence for generation
}


In [ ]:
from tqdm import tqdm
import re
import random

folktales = len(ft_short)
output = ft_short
output['moral_generated'] = ''
output['full_response'] = ''

prefix = "Moral:"
pattern = r"Moral:(.*?)###"
for story in tqdm(range(folktales)):
    n = random.randint(1,118)
    print("\nSTORY",story,"\nSEED", n)
    prompt = ""
    prompt += f"{df['story'].iloc[n-1]}  {prefix} {df['moral'].iloc[n-1]} ### "
    prompt += f"{df['story'].iloc[n]}  {prefix} {df['moral'].iloc[n]} ### "
    prompt += f"{ft_short['text'].iloc[story]}  {prefix} "
    print("PROMPT LENGTH", len(prompt.split()))
    #print(prompt)
    generated = query(prompt, parameters)
    moral = re.search(pattern, generated[-500:]) 
    if moral:
        print("\nMORAL:",moral.group(1))
        output['moral_generated'].iloc[story] = moral.group(1)
        output['full_response'].iloc[story] = generated
    print("~"*77)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
  0%|          | 0/376 [00:00<?, ?it/s]


STORY 0 
SEED 83
PROMPT LENGTH 879


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
  0%|          | 1/376 [00:05<35:16,  5.64s/it]


MORAL:   We are not to be too hard on those who transgress against us, for they are not to blame. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 1 
SEED 25
PROMPT LENGTH 550


  1%|          | 2/376 [00:12<41:17,  6.62s/it]

...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 96 
SEED 57
PROMPT LENGTH 544


 26%|██▌       | 97/376 [10:33<31:35,  6.79s/it]


MORAL:   
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 97 
SEED 33
PROMPT LENGTH 492


 26%|██▌       | 97/376 [15:45<45:18,  9.74s/it]


JSONDecodeError: ignored

In [ ]:
output.head()

,source,nation,title,text,moral_generated,full_response
2,https://fairytalez.com/sharing-joy-and-sorrow/,german,Sharing Joy and Sorrow,"There was once a tailor, who was a quarrelsome...",We are not to be too hard on those who trans...,There was once an Ass whose Master also owned ...
13,https://fairytalez.com/the-peasant-in-heaven/,german,The Peasant in Heaven,"ONCE on a time a poor pious peasant died, and ...",There is no justice in this world.,One bright day in late autumn a family of Ants...
34,https://fairytalez.com/the-laird-o-co/,scottish,The Laird O' Co',"In the days of yore, the proprietors of Colzea...","If you can’t do a kind act, do the next best...",A sacred Image was being carried to the temple...
54,https://fairytalez.com/fool-sold-wisdom/,french,The Fool Who Sold Wisdom,"A fool, in town, did wisdom cry;The people, ea...",A fool is often wiser than a sage.,A Serpent had succeeded in surprising an Eagle...
55,https://fairytalez.com/quarrel-dogs-cats/,french,Quarrel of the Dogs and Cats,"In mansion deck’d with frieze and column,Dwelt...",The end of all quarrels is that they produce...,One day a Stag came to a Sheep and asked her t...


In [ ]:
# interrupted due to error so continue where it left off
prefix = "Moral:"
pattern = r"Moral:(.*?)###"
for story in tqdm(range(97,folktales)):
    n = random.randint(1,117)
    print("\nSTORY",story,"\nSEED", n)
    prompt = ""
    prompt += f"{df['story'].iloc[n-1]}  {prefix} {df['moral'].iloc[n-1]} ### "
    prompt += f"{df['story'].iloc[n]}  {prefix} {df['moral'].iloc[n]} ### "
    prompt += f"{ft_short['text'].iloc[story]}  {prefix} " # leave lesson blank for GPT-J to generate
    print("PROMPT LENGTH", len(prompt.split()))
    #print(prompt)
    generated = query(prompt, parameters)
    moral = re.search(pattern, generated[-500:]) 
    if moral:
        print("\nMORAL:",moral.group(1))
        output['moral_generated'].iloc[story] = moral.group(1)
        output['full_response'].iloc[story] = generated
    print("~"*77)

  0%|          | 0/279 [00:00<?, ?it/s]


STORY 97 
SEED 42
PROMPT LENGTH 551


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
  0%|          | 1/279 [00:09<44:40,  9.64s/it]


MORAL:   The greater the man, the more care he should take of his manners and conduct. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 98 
SEED 12
PROMPT LENGTH 542


  1%|          | 2/279 [00:14<31:34,  6.84s/it]

...

 38%|███▊      | 106/279 [13:03<17:18,  6.00s/it]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 203 
SEED 92
PROMPT LENGTH 494


 38%|███▊      | 107/279 [13:08<21:07,  7.37s/it]


MORAL:   God’s works are good. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 204 
SEED 118


IndexError: ignored

In [ ]:
# interrupted again
prefix = "Moral:"
pattern = r"Moral:(.*?)###"
for story in tqdm(range(204,folktales)):
    n = random.randint(1,117)
    print("\nSTORY",story,"\nSEED", n)
    prompt = ""
    prompt += f"{df['story'].iloc[n-1]}  {prefix} {df['moral'].iloc[n-1]} ### "
    prompt += f"{df['story'].iloc[n]}  {prefix} {df['moral'].iloc[n]} ### "
    prompt += f"{ft_short['text'].iloc[story]}  {prefix} " # Moral blank for GPT-J to generate
    print("PROMPT LENGTH", len(prompt.split()))
    #print(prompt)
    generated = query(prompt, parameters)
    moral = re.search(pattern, generated[-500:]) 
    if moral:
        print("\nMORAL:",moral.group(1))
        output['moral_generated'].iloc[story] = moral.group(1)
        output['full_response'].iloc[story] = generated
    print("~"*77)

  0%|          | 0/172 [00:00<?, ?it/s]


STORY 204 
SEED 36
PROMPT LENGTH 659


  1%|          | 1/172 [00:05<14:19,  5.03s/it]


MORAL:   The wicked are always more afraid of the truth than the truth is of them. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 205 
SEED 26
PROMPT LENGTH 663


  1%|          | 2/172 [00:10<14:22,  5.07s/it]

...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 374 
SEED 23
PROMPT LENGTH 549


 99%|█████████▉| 171/172 [21:25<00:10, 10.31s/it]


MORAL:   When a man has sinned, he must not expect to be happy in the next world. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 375 
SEED 80
PROMPT LENGTH 747


100%|██████████| 172/172 [21:51<00:00,  7.62s/it]


MORAL:   “The world is great, but thou art greater than it!” 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [ ]:
out_csv = output.to_csv('folktales_gptj_375_utf8.csv',encoding='utf-8-sig')

In [ ]:
output.describe()

,source,nation,title,text,moral_generated,full_response
count,376,376,376,376,376,376
unique,376,33,374,375,281,288
top,https://fairytalez.com/sharing-joy-and-sorrow/,french,The Passing of Loku,"The tale of Loku is applied to a large, ugly l...",,
freq,1,95,2,2,89,89


In [ ]:
import numpy as np
ft_blank = output.replace(r'^\s*$', np.nan, regex=True)
ft_blank = ft_blank[ft_blank.isnull().any(axis=1)]
ft_blank.describe()

,source,nation,title,text,moral_generated,full_response
count,96,96,96,96,0,7
unique,96,25,96,96,0,7
top,https://fairytalez.com/mouse-and-mouser/,north_american_native,Mouse and Mouser,"The Mouse went to visit the Cat, and found her...",NaN,"A Dog asleep in a manger filled with hay, was ..."
freq,1,25,1,1,NaN,1


In [ ]:
output = output.replace(r'^\s*$', np.nan, regex=True)
output = output.dropna()

In [ ]:
output.describe()

,source,nation,title,text,moral_generated,full_response
count,280,280,280,280,280,280
unique,280,30,280,280,279,280
top,https://fairytalez.com/sharing-joy-and-sorrow/,french,Sharing Joy and Sorrow,"There was once a tailor, who was a quarrelsome...",A little knowledge is a dangerous thing.,There was once an Ass whose Master also owned ...
freq,1,72,1,1,2,1


In [ ]:
out_csv = output.to_csv('folktales_gptj_280.csv', encoding='utf-8-sig')

#### Prompting using only annotator-approved seed stories

In [ ]:
# obtained from annotator task examination
seeds = [1, 2, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 26, 31, 33, 35, 36, 42, 43, 44, 45, 46, 48, 52, 53, 54, 56, 57, 58, 59, 60, 64, 66, 67, 69, 70, 71, 75, 76, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 91, 92, 93, 95, 96, 97, 98, 99, 100, 102, 103, 104, 106, 108, 109, 110, 111, 112, 113, 114, 116, 117, 118, 120, 121, 122, 123, 127, 128, 129, 130, 131, 133, 135, 137, 141, 143]
n_seeds = len(seeds)
print(n_seeds)

94


In [ ]:
from tqdm import tqdm
import re
import random

folktales = len(ft_short)
output = ft_short
output['moral_generated'] = ''
output['full_response'] = ''

prefix = "Moral:"
pattern = r"Moral:(.*?)###"
for story in tqdm(range(folktales)):
    n1 = random.randint(1,n_seeds)
    n2 = random.randint(1,n_seeds)
    print("\nSTORY",story,"\nSEEDS", n1, n2)
    prompt = ""
    prompt += f"{df['story'].iloc[n1]}  {prefix} {df['moral'].iloc[n1]} ### "
    prompt += f"{df['story'].iloc[n2]}  {prefix} {df['moral'].iloc[n2]} ### "
    prompt += f"{ft_short['text'].iloc[story]}  {prefix} "
    print("PROMPT LENGTH", len(prompt.split()))
    generated = query(prompt, parameters)
    moral = re.search(pattern, generated[-500:]) 
    if moral:
        print("\nMORAL:",moral.group(1))
        output['moral_generated'].iloc[story] = moral.group(1)
        output['full_response'].iloc[story] = generated
    print("~"*77)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
  0%|          | 0/376 [00:00<?, ?it/s]


STORY 0 
SEEDS 68 5
PROMPT LENGTH 723


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
  0%|          | 1/376 [00:05<32:33,  5.21s/it]


MORAL:   We should not beat our wives, but should rather love them and share joy and sorrow with them. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 1 
SEEDS 74 49
PROMPT LENGTH 506


  1%|          | 2/376 [00:16<53:10,  8.53s/it]

...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 374 
SEEDS 94 54
PROMPT LENGTH 522


100%|█████████▉| 375/376 [50:02<00:04,  4.23s/it]


MORAL:   A man may be as good as he pleases, but he must be careful to keep the laws of his country. 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

STORY 375 
SEEDS 71 12
PROMPT LENGTH 640


100%|██████████| 376/376 [50:07<00:00,  8.00s/it]


MORAL:   The fool saith in his heart, There is no God. But the wise man saith, God is.  
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [ ]:
output.head()

,source,nation,title,text,moral_generated,full_response
2,https://fairytalez.com/sharing-joy-and-sorrow/,german,Sharing Joy and Sorrow,"There was once a tailor, who was a quarrelsome...","We should not beat our wives, but should rat...","A Dog asleep in a manger filled with hay, was ..."
13,https://fairytalez.com/the-peasant-in-heaven/,german,The Peasant in Heaven,"ONCE on a time a poor pious peasant died, and ...",,
34,https://fairytalez.com/the-laird-o-co/,scottish,The Laird O' Co',"In the days of yore, the proprietors of Colzea...","If you are a good man, you will be a good ma...","Two Bulls were fighting furiously in a field, ..."
54,https://fairytalez.com/fool-sold-wisdom/,french,The Fool Who Sold Wisdom,"A fool, in town, did wisdom cry;The people, ea...",Those who are wise must not be taken in by t...,A Farmer walked through his field one cold win...
55,https://fairytalez.com/quarrel-dogs-cats/,french,Quarrel of the Dogs and Cats,"In mansion deck’d with frieze and column,Dwelt...",The world is governed by the same laws that ...,The Fox one day thought of a plan to amuse him...


In [ ]:
import numpy as np
output = output.replace(r'^\s*$', np.nan, regex=True)
output = output.dropna()

In [ ]:
output.describe()

,source,nation,title,text,moral_generated,full_response
count,302,302,302,302,302,302
unique,302,29,301,301,301,302
top,https://fairytalez.com/sharing-joy-and-sorrow/,french,The Passing of Loku,"The tale of Loku is applied to a large, ugly l...",A little learning is a dangerous thing.,"A Dog asleep in a manger filled with hay, was ..."
freq,1,79,2,2,2,1


In [ ]:
out_csv = output.to_csv('folktales_gptj_goodseeds.csv', encoding='utf-8-sig')

In [ ]:
randomseed_ft = pd.read_csv("folktales_gptj_280.csv", index_col=0)
goodseed_ft = pd.read_csv("folktales_gptj_goodseeds.csv", index_col=0)

In [ ]:
randomseed_ft = randomseed_ft.rename(columns={'moral_generated':'moral_generated_randomseed'})

In [ ]:
randomseed_ft.head()

,source,nation,title,text,moral_generated_randomseed,full_response
2,https://fairytalez.com/sharing-joy-and-sorrow/,german,Sharing Joy and Sorrow,"There was once a tailor, who was a quarrelsome...",We are not to be too hard on those who trans...,There was once an Ass whose Master also owned ...
13,https://fairytalez.com/the-peasant-in-heaven/,german,The Peasant in Heaven,"ONCE on a time a poor pious peasant died, and ...",There is no justice in this world.,One bright day in late autumn a family of Ants...
34,https://fairytalez.com/the-laird-o-co/,scottish,The Laird O' Co',"In the days of yore, the proprietors of Colzea...","If you can’t do a kind act, do the next best...",A sacred Image was being carried to the temple...
54,https://fairytalez.com/fool-sold-wisdom/,french,The Fool Who Sold Wisdom,"A fool, in town, did wisdom cry;The people, ea...",A fool is often wiser than a sage.,A Serpent had succeeded in surprising an Eagle...
55,https://fairytalez.com/quarrel-dogs-cats/,french,Quarrel of the Dogs and Cats,"In mansion deck’d with frieze and column,Dwelt...",The end of all quarrels is that they produce...,One day a Stag came to a Sheep and asked her t...


In [ ]:
goodseed_ft=goodseed_ft.rename(columns={'moral_generated':'moral_generated_goodseed'})

In [ ]:
goodseed_ft.head()

,source,nation,title,text,moral_generated_goodseed,full_response
2,https://fairytalez.com/sharing-joy-and-sorrow/,german,Sharing Joy and Sorrow,"There was once a tailor, who was a quarrelsome...","We should not beat our wives, but should rat...","A Dog asleep in a manger filled with hay, was ..."
34,https://fairytalez.com/the-laird-o-co/,scottish,The Laird O' Co',"In the days of yore, the proprietors of Colzea...","If you are a good man, you will be a good ma...","Two Bulls were fighting furiously in a field, ..."
54,https://fairytalez.com/fool-sold-wisdom/,french,The Fool Who Sold Wisdom,"A fool, in town, did wisdom cry;The people, ea...",Those who are wise must not be taken in by t...,A Farmer walked through his field one cold win...
55,https://fairytalez.com/quarrel-dogs-cats/,french,Quarrel of the Dogs and Cats,"In mansion deck’d with frieze and column,Dwelt...",The world is governed by the same laws that ...,The Fox one day thought of a plan to amuse him...
61,https://fairytalez.com/mouse-and-mouser/,english,Mouse and Mouser,"The Mouse went to visit the Cat, and found her...","A cat can catch a mouse, but a mouse can cat...",An Ox came down to a reedy pool to drink. As h...


In [ ]:
annotate = pd.concat([goodseed_ft,randomseed_ft], axis=1).dropna()
annotate = annotate.T.drop_duplicates().T
annotate.head()

,source,nation,title,text,moral_generated_goodseed,full_response,moral_generated_randomseed,full_response
2,https://fairytalez.com/sharing-joy-and-sorrow/,german,Sharing Joy and Sorrow,"There was once a tailor, who was a quarrelsome...","We should not beat our wives, but should rat...","A Dog asleep in a manger filled with hay, was ...",We are not to be too hard on those who trans...,There was once an Ass whose Master also owned ...
34,https://fairytalez.com/the-laird-o-co/,scottish,The Laird O' Co',"In the days of yore, the proprietors of Colzea...","If you are a good man, you will be a good ma...","Two Bulls were fighting furiously in a field, ...","If you can’t do a kind act, do the next best...",A sacred Image was being carried to the temple...
54,https://fairytalez.com/fool-sold-wisdom/,french,The Fool Who Sold Wisdom,"A fool, in town, did wisdom cry;The people, ea...",Those who are wise must not be taken in by t...,A Farmer walked through his field one cold win...,A fool is often wiser than a sage.,A Serpent had succeeded in surprising an Eagle...
55,https://fairytalez.com/quarrel-dogs-cats/,french,Quarrel of the Dogs and Cats,"In mansion deck’d with frieze and column,Dwelt...",The world is governed by the same laws that ...,The Fox one day thought of a plan to amuse him...,The end of all quarrels is that they produce...,One day a Stag came to a Sheep and asked her t...
63,https://fairytalez.com/council-held-rats/,french,The Council Held by the Rats,"Old Rodilard, a certain cat,Such havoc of the ...","The man who is wise in council, but a fool i...",It happened once upon a time that a certain Gr...,The man who is to execute an idea must have ...,A Heron was walking sedately along the bank of...


In [ ]:
annotate_out = annotate.to_csv('folktales_gptj_annotate_full.csv', encoding='utf-8-sig')

In [ ]:
annotate.describe()

,source,nation,title,text,moral_generated_goodseed,full_response,moral_generated_randomseed,full_response
count,224,224,224,224,224,224,224,224
unique,224,27,224,224,223,224,224,224
top,https://fairytalez.com/sharing-joy-and-sorrow/,french,Sharing Joy and Sorrow,"There was once a tailor, who was a quarrelsome...",A little learning is a dangerous thing.,"A Dog asleep in a manger filled with hay, was ...",We are not to be too hard on those who trans...,There was once an Ass whose Master also owned ...
freq,1,61,1,1,2,1,1,1


### Evaluation

#### BERTscore



In [ ]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/aesop_gptj_annotate.csv')
df.head()

,Unnamed: 0,story,moral1,moral2,match_moral,match_story
0,0,There was once a little Kid whose growing horn...,Do not let anything turn you from your purpose.,A fool and his money are soon parted.,NaN,NaN
1,1,"The Tortoise, you know, carries his house on h...",Foolish curiosity and vanity often lead to mis...,The best of friends may disagree.,NaN,NaN
2,2,Why in the world do you walk sideways like tha...,Do not tell others how to act unless you can s...,"""How to walk"" is a great mystery.",NaN,NaN
3,3,An Ox came down to a reedy pool to drink. As h...,Do not attempt the impossible.,Do not puff yourself up.,NaN,NaN
4,4,"A Dog and a Cock, who were the best of friends...",Those who try to deceive may expect to be paid...,Do not trust a fox who has a good breakfast in...,NaN,NaN


In [ ]:
# BERTscore maximizes the cosine similarity between contextualized token embeddings from BERT
# https://huggingface.co/spaces/evaluate-metric/bertscore
#!pip install bert_score
#!pip install evaluate
from evaluate import load
bertscore = load("bertscore")
predictions = df['moral2'].tolist()
references = df['moral1'].tolist()
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
print(results)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 3.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

{'precision': [0.694692075252533, 0.7757531404495239, 0.7577666640281677, 0.814271867275238, 0.7021468281745911, 0.6890623569488525, 0.8550083041191101, 0.7586191296577454, 0.679728627204895, 0.79417884349823, 0.7543249130249023, 0.6412128806114197, 0.7229859232902527, 0.7287205457687378, 0.7280494570732117, 0.7164392471313477, 0.7414942383766174, 0.7617347240447998, 0.721060574054718, 0.7636666297912598, 0.7107558250427246, 0.7218910455703735, 0.7245129346847534, 0.7670159935951233, 0.7362233996391296, 0.6836122274398804, 0.6904103755950928, 0.7657789587974548, 0.6999707818031311, 0.72358238697052, 0.7183746099472046, 0.7093199491500854, 0.782828688621521, 0.7662234306335449, 0.7832087874412537, 0.714770495891571, 0.7262613773345947, 0.732566237449646, 0.7863388061523438, 0.7727653980255127, 0.7454982399940491, 0.7322239279747009, 0.7243276834487915, 0.7525192499160767, 0.7161149382591248, 0.6920529007911682, 0.7181766033172607, 0.766592264175415, 0.709810733795166, 0.6910369396209717

In [ ]:
scores = pd.DataFrame.from_dict(results)

In [ ]:
scores

,precision,recall,f1,hashcode,original,generated
0,0.694692,0.640582,0.666541,distilbert-base-uncased_L5_no-idf_version=0.3....,Do not let anything turn you from your purpose.,A fool and his money are soon parted.
1,0.775753,0.751580,0.763475,distilbert-base-uncased_L5_no-idf_version=0.3....,Foolish curiosity and vanity often lead to mis...,The best of friends may disagree.
2,0.757767,0.710920,0.733596,distilbert-base-uncased_L5_no-idf_version=0.3....,Do not tell others how to act unless you can s...,"""How to walk"" is a great mystery."
3,0.814272,0.791208,0.802574,distilbert-base-uncased_L5_no-idf_version=0.3....,Do not attempt the impossible.,Do not puff yourself up.
4,0.702147,0.726721,0.714223,distilbert-base-uncased_L5_no-idf_version=0.3....,Those who try to deceive may expect to be paid...,Do not trust a fox who has a good breakfast in...
...,...,...,...,...,...,...
142,0.764713,0.801538,0.782693,distilbert-base-uncased_L5_no-idf_version=0.3....,Do not give up friends for foes.,Beware of the man who is always ready to serve...
143,0.699244,0.732115,0.715302,distilbert-base-uncased_L5_no-idf_version=0.3....,The wicked deserve no aid.,Do not steal.
144,0.744572,0.692527,0.717607,distilbert-base-uncased_L5_no-idf_version=0.3....,A fool may deceive by his dress and appearance...,Silence is golden.
145,0.740209,0.763522,0.751685,distilbert-base-uncased_L5_no-idf_version=0.3....,A small gain is worth more than a large promise.,The world is full of little fishes who are muc...


In [ ]:
scores['f1'].mean()

0.7252980223318346

#### BLANC

https://aclanthology.org/2020.eval4nlp-1.2.pdf

Result was very poor.

In [ ]:
scores = pd.DataFrame()

In [ ]:
scores['original'] = df['moral1']
scores['generated'] = df['moral2']

In [ ]:
# Fill in the BLANC: Human-free quality estimation of document summaries.
# BLANC is a reference-free approach to the automatic estimation of document summary quality. 
# https://github.com/PrimerAI/blanc
!pip install blanc
from blanc import BlancHelp, BlancTune
from tqdm import tqdm
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 44.7 MB/s 
     |████████████████████████████████| 182 kB 80.4 MB/s 
     |████████████████████████████████| 7.6 MB 60.1 MB/s 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
blanc_help = BlancHelp(device='cuda')
documents = df['story'].tolist()
#documents = df['moral1'].tolist()
#summaries = df['moral1'].tolist()
summaries = df['moral2'].tolist()
blanc_scores = blanc_help.eval_pairs(documents, summaries)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 5800/5800 [16:58<00:00,  5.69it/s]


In [ ]:
#scores['blanc_morals'] = blanc_scores
#scores['blanc_story_m1'] = blanc_scores
scores['blanc_story_m2'] = blanc_scores

In [ ]:
scores.head()

,blanc,original,generated,blanc_morals,blanc_story_m1,blanc_story_m2
0,NaN,Do not let anything turn you from your purpose.,A fool and his money are soon parted.,0.000000,0.013245,0.052980
1,NaN,Foolish curiosity and vanity often lead to mis...,The best of friends may disagree.,0.166667,0.020548,0.027397
2,NaN,Do not tell others how to act unless you can s...,"""How to walk"" is a great mystery.",0.000000,-0.044444,0.088889
3,NaN,Do not attempt the impossible.,Do not puff yourself up.,0.000000,0.014493,0.028986
4,NaN,Those who try to deceive may expect to be paid...,Do not trust a fox who has a good breakfast in...,0.000000,0.021583,0.021583


In [ ]:
scores['blanc_morals'].mean()

0.024614217981564924

In [ ]:
scores['blanc_story_m1'].mean()

0.0012763221640101786

In [ ]:
scores['blanc_story_m2'].mean()

0.011787604902761381

In [ ]:
aesop_blanc = scores.to_csv('aesop_gptj_blanc.csv',encoding='utf-8-sig')

In [ ]:
df = pd.read_csv('drive/MyDrive/folktales_gptj_annotate.csv')
del df['m1_match']
del df['m2_match']
df.head()

,text,moral_generated_goodseed,moral_generated_randomseed
0,"There was once a tailor, who was a quarrelsome...","We should not beat our wives, but should rat...",We are not to be too hard on those who trans...
1,"In the days of yore, the proprietors of Colzea...","If you are a good man, you will be a good ma...","If you can’t do a kind act, do the next best..."
2,"A fool, in town, did wisdom cry;The people, ea...",Those who are wise must not be taken in by t...,A fool is often wiser than a sage.
3,"In mansion deck’d with frieze and column,Dwelt...",The world is governed by the same laws that ...,The end of all quarrels is that they produce...
4,"Old Rodilard, a certain cat,Such havoc of the ...","The man who is wise in council, but a fool i...",The man who is to execute an idea must have ...


In [ ]:
cols=['text','moral_generated_goodseed','moral_generated_randomseed']
ft_scores = df[cols]

In [ ]:
blanc_help = BlancHelp(device='cuda')
documents = df['text'].tolist()
#documents = df['moral_generated_goodseed'].tolist()
#summaries = df['moral_generated_goodseed'].tolist()
summaries = df['moral_generated_randomseed'].tolist()
blanc_scores = blanc_help.eval_pairs(documents, summaries)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 6042/6042 [01:22<00:00, 73.45it/s]


In [ ]:
#ft_scores['blanc_morals'] = blanc_scores
#ft_scores['blanc_text_goodseed'] = blanc_scores
ft_scores['blanc_text_randomseed'] = blanc_scores

In [ ]:
ft_scores.head()

,text,moral_generated_goodseed,moral_generated_randomseed,blanc_morals,blanc_text_randomseed,blanc_text_goodseed
0,"There was once a tailor, who was a quarrelsome...","We should not beat our wives, but should rat...",We are not to be too hard on those who trans...,0.000000,-0.016484,0.038462
1,"In the days of yore, the proprietors of Colzea...","If you are a good man, you will be a good ma...","If you can’t do a kind act, do the next best...",-0.166667,0.006061,-0.024242
2,"A fool, in town, did wisdom cry;The people, ea...",Those who are wise must not be taken in by t...,A fool is often wiser than a sage.,0.200000,0.020202,0.020202
3,"In mansion deck’d with frieze and column,Dwelt...",The world is governed by the same laws that ...,The end of all quarrels is that they produce...,0.000000,0.000000,0.007092
4,"Old Rodilard, a certain cat,Such havoc of the ...","The man who is wise in council, but a fool i...",The man who is to execute an idea must have ...,0.000000,0.013423,0.026846


In [ ]:
ft_scores['blanc_morals'].mean()

0.05392010023706452

In [ ]:
ft_scores['blanc_text_goodseed'].mean()

0.00253566505211153

In [ ]:
ft_scores['blanc_text_randomseed'].mean()

0.001424086517050165

In [ ]:
ft_blanc = ft_scores.to_csv('folktales_gptj_blanc.csv',encoding='utf-8-sig')

In [ ]:
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░▒▒▓▓▓▓████
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒    ░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░▒▒▓▓▓▓▓▓▓▓██
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒      ░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░▒▒▓▓▓▓▓▓▓▓▓▓██
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒        ░░▒▒▒▒▒▒▒▒▓▓▒▒▒▒▒▒▒▒▒▒░░░░▒▒▓▓▓▓▓▓▓▓▓▓▓▓██
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒        ░░▒▒▒▒▒▒▒▒▓▓▒▒▒▒▒▒░░    ░░▒▒▓▓▓▓▓▓▓▓▓▓▒▒██
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒          ▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░  ░░░░▒▒▓▓▓▓▓▓▓▓██▓▓▒▒██
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░▒▒░░  ░░░░░░▓▓▓▓▓▓▓▓████▓▓▒▒██
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▓▓██████▓▓▒▒██
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▓▓██▓▓████▓▓▒▒▓▓██
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▓▓██████▓▓▒▒▒▒▓▓██
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░▒▒░░░░░░░░▒▒████████▓▓▒▒▒▒██▓▓
▒▒▒▒▒▒▒▒░░                    ░░░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒░░░░▒▒▓▓██████▓▓▒▒▒▒▒▒██▒▒
▒▒▓▓▓▓▒▒░░                ░░░░░░░░▒▒▒▒▒▒▓▓▓▓▓▓▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░▒▒▒▒░░▒▒▒▒▓▓████▓▓▒▒▒▒▒▒▓▓██▒▒
▓▓████▓▓░░      ░░░░    ░░░░░░░░▒▒▒▒▒▒▓▓▓▓██▒▒▓▓▓▓▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░▒▒▒▒▒▒▒▒▒▒▓▓▓▓▒▒▒▒▒▒▒▒▓▓████▒▒
▓▓██▓▓▒▒            ░░░░░░░░░░▒▒▒▒▒▒▒▒▒▒▓▓▒▒░░░░▓▓▓▓▓▓▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓██▓▓▒▒
▒▒▓▓▒▒░░            ░░░░░░░░▒▒▒▒▒▒░░▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░▒▒░░▒▒▒▒▓▓▓▓████▓▓▒▒
▒▒▒▒▒▒░░            ░░░░░░░░▒▒▒▒▒▒░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░▒▒▓▓▓▓▓▓▓▓▒▒░░
▒▒▒▒▓▓▒▒░░            ░░  ░░░░░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓▒▒░░
▒▒▒▒▓▓▓▓▒▒░░░░░░      ░░░░            ░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▒▒▒▒░░
▒▒▒▒▓▓████▓▓▒▒▒▒░░░░░░  ░░                ░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓██████████▓▓▓▓▒▒▒▒░░
▒▒▒▒▒▒▓▓▓▓▓▓▓▓▒▒▒▒░░░░░░                    ░░░░░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓██████▓▓▓▓▒▒▒▒▒▒░░
▒▒▒▒▒▒▓▓▓▓▒▒▓▓▓▓▓▓▒▒░░░░░░                    ░░░░░░░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓██▓▓▓▓▒▒▒▒▒▒▒▒░░
▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▒▒░░░░░░  ░░              ░░░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓▓▓▒▒▒▒▒▒▒▒░░
▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▒▒░░░░░░                ░░░░░░░░░░▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓▓▓▓▓▒▒▒▒▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▒▒▒▒▒▒▒▒▒▒▓▓▓▓▒▒░░░░░░░░  ░░        ░░    ░░░░░░▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓▓▓▓▓▒▒▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░    ░░  ░░░░░░░░▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▒▒▒▒▒▒▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░          ░░  ░░░░░░▒▒▓▓▓▓▓▓▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓▓▓▒▒░░░░░░░░░░░░░░░░░░░░░░░░  ░░      ░░░░░░░░░░▒▒▒▒▒▒▓▓▓▓▒▒░░▒▒▒▒▒▒░░░░▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░        ░░░░░░░░▒▒▒▒▒▒▓▓▓▓▒▒▒▒▒▒░░░░░░░░░░░░
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░  ░░░░░░░░░░░░░░  ░░░░░░░░▒▒▒▒▒▒▒▒▓▓▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░  ░░░░░░░░░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░  ░░░░░░░░░░░░▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░▒▒▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  ░░░░░░░░░░░░░░░░░░            ░░░░▒▒▒▒▒▒▒▒▒▒▒▒░░▒▒░░░░░░░░▒▒▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  ░░░░░░░░░░░░              ░░░░░░░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░  ░░░░░░░░              ░░░░░░░░░░░░░░░░░░▒▒▒▒░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░  ░░░░░░░░        ░░    ░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░  ░░░░            ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░    ░░              ░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░    ░░          ░░░░░░  ░░░░░░▒▒░░░░░░░░░░░░░░░░▒▒░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒
▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░              ░░░░      ░░▒▒░░░░░░░░▒▒░░░░░░░░▒▒░░▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
'''moral support fox'''
'''jimmy'''